## 1. Resumen

Este documento proporciona documentación técnica completa para una implementación interactiva del algoritmo de búsqueda de caminos A*. El sistema cuenta con visualización en tiempo real, heurísticas configurables y control de ejecución paso a paso. La implementación demuestra conceptos fundamentales en algoritmos de búsqueda en grafos, geometría computacional y visualización interactiva.

## 2. Fundamento Teórico

### 2.1 Descripción del Algoritmo

A* (A-estrella) es un algoritmo de búsqueda informada que encuentra el camino óptimo entre dos nodos en un grafo ponderado. Combina las ventajas del algoritmo de Dijkstra con la búsqueda guiada por heurística.

### 2.2 Función de Costo

El algoritmo evalúa cada nodo utilizando la función de costo:

$$f(n) = g(n) + h(n)$$

Donde:
- $f(n)$ : Costo total estimado desde el inicio hasta la meta a través del nodo $n$
- $g(n)$ : Costo real desde el nodo inicial hasta el nodo $n$
- $h(n)$ : Costo estimado heurístico desde el nodo $n$ hasta la meta

### 2.3 Extensión A* Ponderado

Esta implementación soporta A* ponderado con un factor de inflación $\epsilon$:

$$f(n) = g(n) + \epsilon \cdot h(n)$$

Propiedades:
- Cuando $\epsilon = 1.0$ : Búsqueda A* óptima
- Cuando $\epsilon > 1.0$ : Búsqueda voraz, más rápida pero potencialmente subóptima
- Cuando $\epsilon \to \infty$ : Se aproxima a búsqueda voraz pura best-first

### 2.4 Función Heurística

Se implementa la heurística de distancia euclidiana:

$$h(n) = \sqrt{(x_n - x_{meta})^2 + (y_n - y_{meta})^2}$$

Esta heurística es:
- **Admisible**: Nunca sobreestima el costo real
- **Consistente**: Satisface la desigualdad triangular

### 2.5 Modelo de Costo de Movimiento

La cuadrícula soporta movimiento en 8 direcciones con costos diferenciales:

- Direcciones cardinales (N, S, E, O): $costo = 1.0$
- Direcciones diagonales (NE, NO, SE, SO): $costo = \sqrt{2} \approx 1.414$

## 3. Pseudocódigo del Algoritmo

```
función A_ESTRELLA(inicio, meta, epsilon):
    conjuntoAbierto = ColaPrioridad()
    conjuntoCerrado = Conjunto()
    
    inicio.g = 0
    inicio.h = heuristica(inicio, meta)
    inicio.f = inicio.g + epsilon * inicio.h
    
    conjuntoAbierto.insertar(inicio)
    
    mientras no conjuntoAbierto.estaVacio():
        actual = conjuntoAbierto.extraerMinimo()
        
        si actual == meta:
            retornar reconstruirCamino(actual)
        
        conjuntoCerrado.agregar(actual)
        
        para cada vecino en actual.vecinos:
            si vecino en conjuntoCerrado:
                continuar
            
            g_tentativo = actual.g + costo(actual, vecino)
            
            si g_tentativo < vecino.g:
                vecino.padre = actual
                vecino.g = g_tentativo
                vecino.h = heuristica(vecino, meta)
                vecino.f = vecino.g + epsilon * vecino.h
                
                si vecino no en conjuntoAbierto:
                    conjuntoAbierto.insertar(vecino)
    
    retornar FALLO
```

## 4. Arquitectura de la Implementación

### 4.1 Componentes Principales

#### 4.1.1 Clase Nodo

La clase `Nodo` representa una celda individual en la cuadrícula:

**Atributos:**
- Espaciales: `fila`, `col`, `x`, `y`, `ancho`
- Grafo: `vecinos` (lista de vecinos)
- Costo: `costo_g`, `costo_h`, `costo_f`
- Camino: `nodo_padre` (referencia al padre)
- Visual: `color` (representación del estado)

**Métodos Principales:**
- `calcular_vecinos()`: Calcula vecinos válidos con costos de movimiento
- `renderizar()`: Renderiza la representación visual del nodo
- Establecedores de estado: `establecer_pared()`, `establecer_visitado()`, etc.

#### 4.1.2 Algoritmo de Búsqueda

La función `buscar_camino_a_estrella()` implementa la lógica central de A*:

**Parámetros:**
- `grid`: Arreglo 2D de objetos Nodo
- `inicio`: Nodo inicial
- `fin`: Nodo meta
- `funcion_dibujar`: Callback para visualización
- `epsilon`: Factor de peso heurístico
- `mostrar_costos`: Booleano para mostrar costos
- `velocidad`: Control de velocidad de ejecución

**Retorna:**
- Lista con información del camino o mensaje de fallo

**Complejidad Temporal:** $O(b^d)$ donde $b$ es el factor de ramificación y $d$ es la profundidad

**Complejidad Espacial:** $O(b^d)$ para almacenar conjuntos abiertos y cerrados

### 4.2 Optimización de Movimiento Diagonal

La implementación incluye prevención de corte de esquinas:

```python
if abs(df) == 1 and abs(dc) == 1:  # Movimiento diagonal
    if not grid[self.fila + df][self.col].es_pared() or \
       not grid[self.fila][self.col + dc].es_pared():
        self.vecinos.append((vecino, costo))
```

Esto asegura que el movimiento diagonal está bloqueado si ambas celdas cardinales adyacentes son paredes, previniendo búsqueda de caminos poco realista a través de esquinas.

## 5. Parámetros de Configuración

### 5.1 Configuración de Pantalla

| Parámetro | Valor | Descripción |
|-----------|-------|-------------|
| `ANCHO_BASE` | 1600px (máx) | Ancho base de ventana |
| `ALTO_BASE` | 1000px (máx) | Alto base de ventana |
| `PANEL_INFO` | 250px | Ancho del panel de información |
| `FILAS` | 11 | Dimensiones de cuadrícula por defecto |

### 5.2 Parámetros del Algoritmo

| Parámetro | Rango | Por Defecto | Efecto |
|-----------|-------|-------------|--------|
| `epsilon` | [1.0, 5.0] | 1.2 | Peso heurístico |
| `velocidad` | [0, 20] | 5 | Nodos por actualización de visualización |
| `mostrar_costos` | Booleano | True | Mostrar costos de nodos |

### 5.3 Esquema de Colores

La implementación usa una paleta de colores inspirada en cyberpunk:

- Fondo: RGB(8, 8, 20)
- Nodo Inicio: RGB(0, 255, 255) - Cian
- Nodo Final: RGB(255, 0, 255) - Magenta
- Camino: RGB(0, 255, 150) - Verde
- Visitado: RGB(80, 0, 100) - Púrpura Oscuro
- Frontera: RGB(255, 150, 0) - Naranja
- Procesando: RGB(255, 255, 0) - Amarillo

## 6. Interacción del Usuario

### 6.1 Controles del Mouse

- **Clic Izquierdo**: Colocar nodo inicio (primero), nodo final (segundo), luego paredes
- **Clic Derecho**: Eliminar nodos
- **Rueda del Mouse**: Desplazar salida de consola

### 6.2 Controles del Teclado

| Tecla | Función | Descripción |
|-------|---------|-------------|
| ENTER | Ejecutar | Iniciar algoritmo de búsqueda de caminos |
| ESPACIO | Paso | Avanzar un paso (modo manual) |
| P | Pausa | Alternar estado de pausa |
| R | Reiniciar | Limpiar cuadrícula y reiniciar |
| M | Métricas | Alternar visualización de costos |
| W/S | Velocidad | Aumentar/disminuir velocidad de ejecución |
| A/D | Epsilon | Disminuir/aumentar valor epsilon |

## 7. Análisis de Rendimiento

### 7.1 Complejidad Computacional

Para una cuadrícula de tamaño $n \times n$:

**Complejidad Temporal:**
- Mejor Caso: $O(d)$ donde $d$ es la longitud del camino directo
- Caso Promedio: $O(b \log n)$ con operaciones de cola de prioridad
- Peor Caso: $O(n^2 \log n)$ explorando toda la cuadrícula

**Complejidad Espacial:**
- Almacenamiento de Cuadrícula: $O(n^2)$
- Conjunto Abierto: $O(n^2)$ en el peor caso
- Conjunto Cerrado: $O(n^2)$ en el peor caso

### 7.2 Impacto de Epsilon

Relación empírica entre $\epsilon$ y rendimiento:

$$\text{Ganancia de Velocidad} \approx 1 + \frac{(\epsilon - 1)}{2}$$

$$\text{Límite de Suboptimalidad} \leq \epsilon$$

Esto significa que los caminos encontrados con $\epsilon = 1.5$ están garantizados a no ser peores que 1.5 veces el costo del camino óptimo.

## 8. Características de Visualización

### 8.1 Visualización de Estado en Tiempo Real

El sistema proporciona:
- Coordenadas del nodo en procesamiento actual
- Número de nodos explorados
- Tamaño del conjunto abierto (frontera)
- Modo de ejecución (MANUAL/AUTO/PAUSADO)

### 8.2 Visualización de Costos

Cuando está habilitado, cada nodo muestra:
- $g(n)$: Costo real desde el inicio (arriba)
- $f(n)$: Costo total estimado (abajo)

Requisitos:
- El ancho del nodo debe exceder 40 píxeles
- El nodo debe estar en estado frontera, visitado o camino

## 9. Ejemplo de Implementación

### 9.1 Patrón de Uso Básico

```python
# Inicializar cuadrícula
FILAS = 11
grid = inicializar_grilla(FILAS, ancho_disponible)

# Configurar parámetros del algoritmo
epsilon = 1.2  # Ligera preferencia por velocidad sobre optimalidad
velocidad = 5  # Actualizar visualización cada 5 nodos

# Establecer inicio y meta
inicio = grid[0][0]
fin = grid[10][10]

# Calcular vecinos para todos los nodos
for fila in grid:
    for nodo in fila:
        nodo.calcular_vecinos(grid)

# Ejecutar búsqueda
resultado = buscar_camino_a_estrella(
    grid, inicio, fin, funcion_dibujar,
    epsilon, mostrar_costos, velocidad
)
```

### 9.2 Reconstrucción de Camino

Una vez que se alcanza la meta, el camino se reconstruye siguiendo los punteros padre:

```python
def construir_ruta(inicio, fin, funcion_dibujar):
    camino = []
    nodo_actual = fin
    
    while nodo_actual != inicio:
        nodo_actual = nodo_actual.nodo_padre
        if nodo_actual != inicio:
            nodo_actual.establecer_camino()
            camino.insert(0, nodo_actual)
    
    return [inicio] + camino + [fin]
```

Esto produce un camino completo desde el inicio hasta la meta, visualizado en tiempo real.

## 10. Garantías de Optimalidad

### 10.1 Propiedades de A* Estándar

Con $\epsilon = 1.0$ y una heurística admisible:

**Teorema 1 (Optimalidad):** A* encuentra el camino óptimo si existe.

**Esbozo de Prueba:** 
1. A* expande nodos en orden de $f(n) = g(n) + h(n)$
2. Si $h(n)$ es admisible, entonces $f(n) \leq f^*(n)$ donde $f^*$ es el costo óptimo real
3. Cuando se expande la meta, todos los demás caminos tienen $f(n) \geq f^*(meta)$
4. Por lo tanto, el camino encontrado es óptimo

### 10.2 Límites de A* Ponderado

Para A* ponderado con $\epsilon > 1.0$:

**Teorema 2 (Suboptimalidad Acotada):** 
Si $C^*$ es el costo óptimo y $C$ es el costo encontrado por A* ponderado, entonces:

$$C \leq \epsilon \cdot C^*$$

Esto proporciona una garantía de calidad incluso cuando se sacrifica optimalidad por velocidad.

## 11. Modos de Ejecución

### 11.1 Modo Manual

- `velocidad = 0`
- El usuario presiona ESPACIO para avanzar cada paso
- Ideal para propósitos educativos y comprensión del algoritmo

### 11.2 Modo Automático

- `velocidad > 0`
- La visualización se actualiza cada N nodos
- Valores más altos aumentan la velocidad pero reducen la visibilidad

### 11.3 Función de Pausa

- Presionar P para pausar/reanudar en cualquier momento
- El estado se preserva durante la pausa
- Útil para analizar estados intermedios

## 12. Especificaciones Técnicas

### 12.1 Dependencias

```python
import pygame        # Gráficos y manejo de eventos
import math          # Operaciones matemáticas
from queue import PriorityQueue  # Gestión eficiente de conjunto abierto
```

### 12.2 Requisitos del Sistema

- Python 3.7+
- Pygame 2.0+
- Mínimo 4GB RAM
- Resolución de pantalla: 1024x768 o superior

### 12.3 Métricas de Rendimiento

En una cuadrícula estándar (11x11) con densidad moderada de obstáculos:

- Nodos explorados promedio: 40-60
- Tiempo de ejecución promedio: 100-200ms
- Uso de memoria: ~5MB
- Tasa de fotogramas: 60 FPS mantenidos durante la visualización

## 13. Extensiones y Trabajo Futuro

### 13.1 Mejoras Potenciales

1. **Heurísticas Alternativas**
   - Distancia Manhattan: $h(n) = |x_n - x_{meta}| + |y_n - y_{meta}|$
   - Distancia Chebyshev: $h(n) = \max(|x_n - x_{meta}|, |y_n - y_{meta}|)$

2. **Búsqueda Bidireccional**
   - Buscar simultáneamente desde inicio y meta
   - Aceleración teórica: $O(b^{d/2})$ vs $O(b^d)$

3. **Búsqueda de Punto de Salto**
   - Optimizar para cuadrículas de costo uniforme
   - Reducir expansiones de nodos podando caminos simétricos

4. **Costo Variable de Terreno**
   - Implementar celdas ponderadas
   - Modelar diferentes tipos de terreno